##  GR DUSP1 Gating Notebook

The Purpose of this notebook is:
1) Load in all analyisis for final dataframe preparation (GR_Confirmation)
3) Filter GR data to remove partial cells
4) Estimate GR cytoplasmic area from DUPS1 data
5) GR intensity to molecular counts 
6) Concatonate final GR and DUSP1 dataframes

In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import dask.array as da
import os
import sys
import logging
import seaborn as sns
import datetime
import glob

# Today's date
today = datetime.date.today()
# Format date as 'Jun03' (for example)
date_str = today.strftime("%b%d")

logging.getLogger('matplotlib.font_manager').disabled = True
numba_logger = logging.getLogger('numba')
numba_logger.setLevel(logging.WARNING)

matplotlib_logger = logging.getLogger('matplotlib')
matplotlib_logger.setLevel(logging.WARNING)

src_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
print(src_path)
sys.path.append(src_path)

from src.Analysis_DUSP1 import DUSP1DisplayManager, PostProcessingPlotter, ExperimentPlotter

In [ ]:
# Base directory containing your CSV files
base_dir = "/Volumes/share/Users/Eric/GR_DUSP1_AllData/DUSP1_FinalAnalysis_062425"
save_dir = "/Volumes/share/Users/Eric/GR_DUSP1_AllData/DUSP1_FinalAnalysis_062425/ConcatPlots"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [ ]:
# Helper to load and concat by pattern
def load_and_concat(pattern):
    paths = glob.glob(os.path.join(base_dir, pattern))
    dfs = []
    for path in paths:
        df = pd.read_csv(path)
        # normalize columns to lowercase
        df.columns = [c.lower() for c in df.columns]
        dfs.append(df)
    return pd.concat(dfs, ignore_index=True)

In [ ]:
# Load each DUSP1 dataset
ssit_all      = load_and_concat("*_SSITcellresults.csv")
spots_all     = load_and_concat("*_FinalSpots.csv")
clusters_all  = load_and_concat("*_FinalClusters.csv")
cellprops_all = load_and_concat("*_FinalCellProps.csv")

# Extract replica letter from strings like "D_slide1", "E_day2", etc.
for df in [ssit_all, spots_all, clusters_all, cellprops_all]:
    df['replica'] = df['replica'].str.extract(r'^([D-N])_')

# Optional: warn if any entries didn’t match
for name, df in zip(
    ['SSIT', 'Spots', 'Clusters', 'CellProps'],
    [ssit_all, spots_all, clusters_all, cellprops_all]
):
    n_missing = df['replica'].isna().sum()
    if n_missing > 0:
        print(f"Warning: {n_missing} entries in {name} did not match replica pattern.")

# Quick check on length of each dataset
print(f"SSIT cells: {len(ssit_all)}")
print(f"Spots:      {len(spots_all)}")
print(f"Clusters:   {len(clusters_all)}")
print(f"Cell props: {len(cellprops_all)}")

In [ ]:
def add_replica_prefix(df: pd.DataFrame,
                       replica_col: str = 'replica',
                       id_col: str = 'unique_cell_id',
                       prefix_map: dict[str, int] = None) -> pd.DataFrame:
    """
    For each replica in prefix_map, offset its unique_cell_id by
      prefix * (10 ** num_digits)
    where num_digits is the number of digits in that replica’s max ID.

    Parameters
    ----------
    df : pd.DataFrame
        Your dataframe containing replica_col and id_col.
    replica_col : str
        Name of column holding replica labels (e.g. 'A','B','C').
    id_col : str
        Name of the integer column to rewrite.
    prefix_map : dict[str,int]
        Mapping of replica → small integer prefix.
        E.g. {'A':1, 'B':2, 'C':3}.

    Returns
    -------
    pd.DataFrame
        The same df, with id_col updated in place.
    """
    if prefix_map is None:
        raise ValueError("You must supply a prefix_map, e.g. {'A':1,'B':2,'C':3}")

    for rep, prefix in prefix_map.items():
        mask = df[replica_col] == rep
        if not mask.any():
            continue

        # compute number of digits based on this replica’s max ID
        max_id = int(df.loc[mask, id_col].max())
        num_digits = len(str(max_id))

        offset = prefix * (10 ** num_digits)
        df.loc[mask, id_col] = df.loc[mask, id_col] + offset

    return df

In [ ]:
# Load in the GR data
gr_data = pd.read_csv('/Users/ericron/Desktop/AngelFISH/Publications/Ron_2024/dataframes/GR_ALL_noIC_pregate.csv')

In [ ]:
# define the prefixes you want:
prefix_map = {'A':150, 'B':160, 'C':170}

# apply the function
gr_data = add_replica_prefix(gr_data,
                        replica_col='replica',
                        id_col='unique_cell_id',
                        prefix_map=prefix_map)

In [ ]:
# 2) FIT POLYNOMIAL TO (NUC, CYTO) FROM DUSP1_ALL
# =========================
# We'll use only the rows that have valid nuc_area_px and cyto_area_px.


x_nuc = ssit_all['nuc_area'].values
y_cyto = ssit_all['cyto_area'].values

# Fit a 2nd-degree polynomial: cyto_area_px = a*(nuc_area_px)^2 + b*(nuc_area_px) + c
poly_coeffs = np.polyfit(x_nuc, y_cyto, deg=2)

# optional: for debugging/inspection
print("Fitted polynomial coefficients (a, b, c):", poly_coeffs)
# plot the fitted polynomial on the data
plt.scatter(x_nuc, y_cyto, label='data')
x_fit = np.linspace(x_nuc.min(), x_nuc.max(), 100)
y_fit = np.polyval(poly_coeffs, x_fit)
plt.plot(x_fit, y_fit, label='fitted polynomial', color='red')
plt.xlabel('Nuclear area (px)')
plt.ylabel('Cytoplasm area (px)')
plt.show()


# 3) ESTIMATE CYTO AREA IN GR_ALL
# =========================
# We'll store the computed cytoplasm area in a new column: 'CalcCytoArea'
# Evaluate the polynomial at GR_ALL['nuc_area'].
print('Predict the cyto area of GR from dusp1')
gr_data['CalcCytoArea'] = np.polyval(poly_coeffs, gr_data['nuc_area'])

# 4) GATE BOTH DATAFRAMES ON [25%, 75%] NUCLEAR AREA
# =========================
# We'll define a helper function for gating.
num_cells = ssit_all.shape[0]
def gate_on_nuc_area(df, nuc_col):
    """Return a copy of df gated to [25th, 75th percentile] of nuc_col."""
    lower = df[nuc_col].quantile(0.25)
    upper = df[nuc_col].quantile(0.75)
    return df[(df[nuc_col] >= lower) & (df[nuc_col] <= upper)].copy()

# Gate DUSP1_ALL on nuc_area_px
print('+++ Gating Nuc Area +++')
df_dusp_gated = gate_on_nuc_area(ssit_all, 'nuc_area') 

# Gate GR_ALL on nuc_area
df_gr_gated = gate_on_nuc_area(gr_data, 'nuc_area')

print(f"DUSP1_ALL original: {len(ssit_all)} rows -> gated: {len(df_dusp_gated)} rows")
print(f"GR_ALL original:    {len(gr_data)} rows -> gated: {len(df_gr_gated)} rows")

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.histplot(ssit_all['nuc_area'], bins=30, color='skyblue', edgecolor='black')
plt.title("Nuclear Area (Before Gating)")
plt.subplot(1, 2, 2)
sns.histplot(df_dusp_gated['nuc_area'], bins=30, color='orange', edgecolor='black')
plt.title("Nuclear Area (After Gating)")
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.histplot(gr_data['nuc_area'], bins=30, color='skyblue', edgecolor='black')
plt.title("Nuclear Area (Before Gating) IC")
plt.subplot(1, 2, 2)
sns.histplot(df_gr_gated['nuc_area'], bins=30, color='orange', edgecolor='black')
plt.title("Nuclear Area (After Gating) IC ")
plt.tight_layout()
plt.show()


# # (Optional) Save a copy of the gated GR data
# df_gr_gated.to_csv("GR_ALL_Gated_On_Nuc.csv", index=False)

nuc_mean = df_dusp_gated['nuc_area'].mean()
cyto_mean = df_dusp_gated['cyto_area_px'].mean()
ratio = nuc_mean / cyto_mean
print("Estimated (Nuc : Cyto) area ratio =", ratio)

# Remove timepoints 20, 40, 60, 90, 150 without replicas
timepoints_to_remove = [20, 40, 60, 90, 150]
df_gr_gated = df_gr_gated[~df_gr_gated['time'].isin(timepoints_to_remove)]

# # (Optional) Randomly assign Dex=0 rows to 1, 10, 100 nM
# # Find rows where Dex_Conc == 0
# mask_zero = (df_gr_gated['Dex_Conc'] == 0)
# n_zero = mask_zero.sum()
# if n_zero > 0:
#     # Generate random indices in [0, 2] for each zero row
#     rand_indices = np.random.randint(0, 3, size=n_zero)
#     dex_values = [1, 10, 100]
#     df_gr_gated.loc[mask_zero, 'Dex_Conc'] = [dex_values[i] for i in rand_indices]


# 5) COMPUTE "NORMALIZED" GR FOR NUC & CYTO IN GR_ALL
# =========================
#  Define bins & threshold:
binsCyt = 30
threshold = 0.01

#  Sort cytoplasmic intensities:
sortGRcyt = df_gr_gated['cytoGRint'].sort_values().values

#  Identify the indices for the 1% and 99% cutoffs
low_idx  = int(np.ceil(threshold * len(sortGRcyt))) - 1
high_idx = int(np.ceil((1 - threshold) * len(sortGRcyt))) - 1


#  Gather the min/max thresholds
minThreshold = sortGRcyt[low_idx]
maxCytGR     = sortGRcyt[high_idx]


def apply_cyt_norm(values):
    # Scale to [0,1] based on [minThreshold, maxCytGR], then clip
    scaled = (values - minThreshold) / (maxCytGR - minThreshold)
    clipped = np.clip(scaled, 0, 1)
    return np.round(clipped * binsCyt).astype(int)

# Compute normGRcyt:
df_gr_gated['normGRcyt'] = apply_cyt_norm(df_gr_gated['cytoGRint'])

# Brian's Matlab logic for the nuclear intensity:
#   'nucGRLevel = ratio * Nuc_GR_avg_int', then scale by the same [minThreshold, maxCytGR].
df_gr_gated['nucGR_Level'] = ratio * df_gr_gated['nucGRint']
df_gr_gated['normGRnuc']   = apply_cyt_norm(df_gr_gated['nucGR_Level'])

# Remove intermediate column:
df_gr_gated.drop(columns=['nucGR_Level'], inplace=True)

# Debug: check a few rows
print(df_gr_gated[['nucGRint','cytoGRint','normGRnuc','normGRcyt']].head())

# Save the gated GR data with normalized GR columns
df_gr_gated.to_csv(os.path.join(save_dir, f"GR_ALL_Gated_Normed_{date_str}_Final.csv"), index=False)

# Save the gated DUSP1_ALL data
df_dusp_gated.to_csv(os.path.join(save_dir, f"DUSP1_ALL_Gated_{date_str}_Final.csv"), index=False)

In [ ]:
# Compute quartiles on 'nuc_area'
q_low  = cellprops_all['nuc_area'].quantile(0.25)
q_high = cellprops_all['nuc_area'].quantile(0.75)

# Filter cellprops by nuclear area range
gated_cells = cellprops_all[(cellprops_all['nuc_area'] >= q_low) &
                             (cellprops_all['nuc_area'] <= q_high)].copy()


# Extract unique_cell_ids for gating
gated_ids = gated_cells['unique_cell_id'].unique()

# Subset SSIT and clusters by gated unique_cell_id
ssit_gated     = ssit_all[ssit_all['unique_cell_id'].isin(gated_ids)].copy()
clusters_gated = clusters_all[clusters_all['unique_cell_id'].isin(gated_ids)].copy()
spots_gated    = spots_all[spots_all['unique_cell_id'].isin(gated_ids)].copy()

display(gated_cells.shape, ssit_gated.shape, clusters_gated.shape, spots_gated.shape)

# Print cells before and after gating
print(f"Cells before gating: {cellprops_all['unique_cell_id'].nunique()}")
print(f"Cells after gating: {gated_cells['unique_cell_id'].nunique()}")
# Print spots before and after gating
print(f"Spots before gating: {spots_all['unique_cell_id'].nunique()}")
print(f"Spots after gating: {spots_gated['unique_cell_id'].nunique()}")
# Print clusters before and after gating
print(f"Clusters before gating: {clusters_all['unique_cell_id'].nunique()}")
print(f"Clusters after gating: {clusters_gated['unique_cell_id'].nunique()}")

# Save the gated data
gated_cells.to_csv(os.path.join(save_dir, f'gated_cells_{date_str}.csv'), index=False)
ssit_gated.to_csv(os.path.join(save_dir, f'ssit_gated_{date_str}.csv'), index=False)
clusters_gated.to_csv(os.path.join(save_dir, f'clusters_gated_{date_str}.csv'), index=False)
spots_gated.to_csv(os.path.join(save_dir, f'spots_gated_{date_str}.csv'), index=False)

In [ ]:
# Load the gated data (if already saved)
df_dir = "/Volumes/share/Users/Eric/GR_DUSP1_AllData/DUSP1_AllAnalysis_062425/ConcatPlots"
# gated_cells = pd.read_csv(os.path.join(save_dir, f'gated_cells_{date_str}.csv'))
ssit_gated = pd.read_csv(os.path.join(df_dir, f'ssit_gated_Jun24.csv'))
# clusters_gated = pd.read_csv(os.path.join(save_dir, f'clusters_gated_{date_str}.csv'))
# spots_gated = pd.read_csv(os.path.join(save_dir, f'spots_gated_{date_str}.csv'))

In [ ]:
ssit_gated.keys() # Display keys of the gated SSIT data

In [ ]:
# print the number of unique cell IDs in the gated SSIT data
print(f"Unique cell IDs in gated SSIT data: {ssit_gated['unique_cell_id'].nunique()}")

In [ ]:
ssit_gated['replica'].unique()  # Display unique replicates in the gated SSIT data

In [ ]:
plotter = ExperimentPlotter(ssit_gated)

# 1) 100 nM Time Sweep:
plotter.plot_experiment(
    replicas=['D','E','F','M','N'],
    times=[10,20,30,40,50,60,75,90,120,150,180],
    concs=[100],
    save_dir=save_dir
)

# 2) 75 min Conc Sweep:
plotter.plot_experiment(
    replicas=['G','H','I'],
    times=[75],
    concs=[0.001,0.01,0.1,1,10,100,1000,10000],
    save_dir=save_dir
)

# 3) Both‐varying: 0.3,1,10 nM across multiple times:
plotter.plot_experiment(
    replicas=['J','K','L'],
    times=[30,50,75,90,120,180],
    concs=[0.3,1,10],
    save_dir=save_dir
)

In [ ]:
## Cytoplasmic and Nuclear mRNA Spot Counts at 0 min

# Make a copy of the DUSP1 data
DUSP1_data = ssit_gated.copy()

# Subset data for 0 min time points
df_0min = DUSP1_data[DUSP1_data['time'] == 0]

# Plot distribution of nuclear and cytoplasmic mRNA spots across replicas for 0 min time point
fig, axes = plt.subplots(1, 2, figsize=(14, 6), sharey=True)

sns.boxplot(data=df_0min, x='replica', y='num_nuc_spots', ax=axes[0])
axes[0].set_title('Nuclear mRNA Spot Counts at 0 min')
axes[0].set_ylabel('Nuclear Spot Count')
axes[0].set_xlabel('Replica')

sns.boxplot(data=df_0min, x='replica', y='num_cyto_spots', ax=axes[1])
axes[1].set_title('Cytoplasmic mRNA Spot Counts at 0 min')
axes[1].set_ylabel('Cytoplasmic Spot Count')
axes[1].set_xlabel('Replica')

plt.tight_layout()
plt.show()